In [ ]:
import numpy as np

from dosipy.field import poynting
from dosipy.utils.derive import holoborodko
from dosipy.utils.integrate import elementwise_dblquad
from dosipy.utils.dataloader import load_antenna_el_properties

from utils import sph2cart, sph_normals

In [ ]:
head_breadth_adult = {3: [13.8987, 15.2000, 16.5013],
                      5: [13.6776, 14.2954, 15.2000, 16.1046, 16.7224],
                      7: [13.6055, 13.9542, 14.5182, 15.2000, 15.8818, 16.4458, 16.7945]}
head_breadth_child = {3: [10.4240, 11.4000, 12.3760],
                      5: [10.2582, 10.7215, 11.4000, 12.0785, 12.5418],
                      7: [10.2041, 10.4657, 10.8886, 11.4000, 11.9114, 12.3343, 12.5959]}
p = [3, 5, 7]  # number of collocation points
f = [2.4, 3.0, 3.5, 5.0, 6.0, 10.0]  # frequency
d = [2.5, 5, 10, 50, 150]  # separation distance
deg = 33  # degree of integration
edge_length = 0.02
A_pln = edge_length ** 2

In [ ]:
def IPD(r, f, d, deg):
    # source
    data = load_antenna_el_properties(f)
    xs = data.x.to_numpy()
    xs -= xs.max() / 2
    ys = np.zeros_like(xs) + d
    zs = np.zeros_like(xs)
    Is = np.abs(data.ireal.to_numpy() + 1j * data.iimag.to_numpy())
    dx = xs[1] - xs[0]
    Is_x = holoborodko(Is, dx)
    
    # planar target
    N = 33
    x = np.linspace(-edge_length/2, edge_length/2, N)
    z = np.linspace(-edge_length/2, edge_length/2, N)
    X, Z = np.meshgrid(x, z)
    xt_pln = X.ravel()
    yt_pln = np.zeros_like(xt_pln)
    zt_pln = Z.ravel()
    
    nx_pln = 0
    ny_pln = -1
    nz_pln = 0
    n_len_pln = np.sqrt(nx_pln ** 2 + ny_pln ** 2 + nz_pln ** 2)
    
    S_pln = np.empty_like(xt_pln)
    for idx, (xt, yt, zt) in enumerate(zip(xt_pln, yt_pln, zt_pln)):
        Sx, Sy, Sz = poynting(xt, yt, zt, xs, ys, zs, f, Is, Is_x)
        S_pln[idx] = Sx.real * nx_pln + Sy.real * ny_pln + Sz.real * nz_pln
    sPDn_pln = 1 / (2 * A_pln) * elementwise_dblquad(points=np.c_[xt_pln, zt_pln],
                                                     values=S_pln/n_len_pln,
                                                     degree=deg)
    
    # spherical target
    alpha = 2 * np.arcsin(edge_length/2/r)
    N = 33
    theta = np.linspace(np.pi/2 - alpha/2, np.pi/2 + alpha/2, N)
    phi = np.linspace(np.pi-alpha/2, np.pi+alpha/2, N)
    Theta, Phi = np.meshgrid(theta, phi)
    yt_sph, xt_sph, zt_sph = sph2cart(r, Theta.ravel(), Phi.ravel())
    yt_sph -= yt_sph.min()
    
    ny_sph, nx_sph, nz_sph = sph_normals(r, Theta.ravel(), Phi.ravel())
    n_len_sph = np.sqrt(nx_sph ** 2 + ny_sph ** 2 + nz_sph ** 2)
    
    A_sph = elementwise_dblquad(points=np.c_[xt_sph, zt_sph],
                                values=np.sin(Theta.ravel())*r**2/n_len_sph,
                                degree=deg)
    
    S_sph = np.empty_like(xt_sph)
    for idx, (xt, yt, zt) in enumerate(zip(xt_sph, yt_sph, zt_sph)):
        Sx, Sy, Sz = poynting(xt, yt, zt, xs, ys, zs, f, Is, Is_x)
        S_sph[idx] = Sx.real * nx_sph[idx] + Sy.real * ny_sph[idx] + Sz.real * nz_sph[idx]
    sPDn_sph = 1 / (2 * A_sph) * elementwise_dblquad(points=np.c_[xt_sph, zt_sph],
                                                     values=S_sph/n_len_sph,
                                                     degree=deg)
    return sPDn_pln, sPDn_sph